# Warehouse Problem - Lagrangian relaxation

Consider the MIP formulation:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c} & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

In [6]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Excercices

#### 1. Find the lagrangian relaxation formulation where the constraints 2 are relaxed.

\begin{align*}
\max_{\lambda_c} ~ \min \qquad \sum_w c_w x_w +& \sum_{w ,c} t_{w,c} y_{w,c} + \sum_c \lambda_c (1 - \sum_w y_{w,c}) & \\
\text{subject to:} \\
y_{w,c} \leq x_w && \forall w,c \\
\sum_c y_{w,c} \leq C_w  & \forall w \\
x_w, y_{w,c}, z_c \in \mathbb{B} && \forall w,c
\end{align*}

#### 2. Implement the lagrangian relaxation.

In [7]:
from docplex.mp.model import Model

def lagrangian_relaxation(rlambda=[fixed]*nbStores):
    mdl = Model(name='Lagrangian')
    x = mdl.binary_var_list(nbWarehouses,name='open')
    y = mdl.binary_var_matrix(nbStores, nbWarehouses, name='supplier')
    
    for w in Warehouses:
        for s in Stores:
            mdl.add_constraint(y[s,w] <= x[w], 'supply_%d_%d' % (s, w))
    for w in Warehouses:
        mdl.add_constraint(sum(y[s,w] for s in Stores) <= capacity[w] * x[w], 'open_%d' % w)
    
    mdl.minimize(fixed * sum(x) + sum(supplyCost[s][w] * y[s,w]  for s in Stores for w in Warehouses)
                 + sum(rlambda[s] * (1 - sum(y[s,w] for w in Warehouses)) for s in Stores))       
   
    msol = mdl.solve()
    relaxation = [(1 - sum(y[s,w].solution_value for w in Warehouses)) for s in Stores]
    
    return (mdl.objective_value,  # the objective value
            msol, # solution
            relaxation)  # slack of the relaxed constraints

#### Try the function

In [8]:
obj, msol, relaxed = lagrangian_relaxation()
print(msol, relaxed)

solution for: Lagrangian
objective: 282
open_1=1
open_2=1
supplier_0_1=1
supplier_0_2=1
supplier_1_1=1
supplier_5_1=1
supplier_6_1=1
supplier_7_2=1
 [-1.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 1, 1]


#### 3. Implement the Cutting Plane Method

1. State the master problem
2. Keep track of the best feasible solution
3. Use the solution to the master problem to solve the Lagrangian subproblem

In [9]:
rlambda = [fixed]*nbStores
fsol = None
iteration = 0
step = 1
UB = M*nbStores
bestFeasibleValue = UB
bestFeasible = 0

mdl = Model(name='lagrangian')
u = mdl.continuous_var_list(nbStores, name='lambda', ub=M)
v = mdl.continuous_var(name='lagrangian_dual')
mdl.maximize(v)

while True:
    # 1. Compute the lagrangian relaxation
    obj, fsol, relaxation = lagrangian_relaxation(rlambda)
    print("relaxation:",relaxation)
    
    # 2. Check if feasible: all slack should be null
    n_violations = sum(int(abs(relaxation[s])) for s in Stores)
    print('iteration={:d}, obj={:.2f}, n violations={:d}, UB={:.2f}'.format(
        iteration, obj, n_violations, UB))
    if n_violations == 0 and obj < bestFeasibleValue:
        bestFeasible = fsol
        bestFeasibleValue = obj
    if UB - obj < 1e-5:
        break
    iteration+= 1
    
    # 3. Update the lagrangian
    c = obj - sum(rlambda[s] * relaxation[s] for s in Stores)
    print(' --- ')
    print(c)
    mdl.add_constraint(v <= c + sum(relaxation[s] * u[s] for s in Stores))
    msol = mdl.solve()
    rlambda = [u[s].solution_value for s in Stores]
    print(rlambda)
    UB = mdl.objective_value

print('========================')
print('Lagrangian dual: ' + str(UB))
if bestFeasible != 0:
    print('-- best feasible --')
    print(bestFeasible)

relaxation: [-1.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 1, 1]
iteration=0, obj=282.00, n violations=6, UB=3000.00
 --- 
162.0
[0, 0, 300.0, 300.0, 300.0, 0, 0, 0, 300.0, 300.0]
relaxation: [1, 1, 1, -3.0, -2.0, 1, 1, 1, -1.0, -1.0]
iteration=1, obj=-1103.00, n violations=13, UB=1662.00
 --- 
697.0
[0, 300.0, 300.0, 0, 178.33333333333337, 300.0, 300.0, 300.0, 300.0, 300.0]
relaxation: [1, 0.0, 1, 1, 1, -2.0, -2.0, -1.0, -1.0, 1]
iteration=2, obj=-535.67, n violations=11, UB=1240.33
 --- 
486.0
[288.0, 300.0, 300.0, 0, 170.33333333333331, 0, 0, 300.0, 300.0, 300.0]
relaxation: [-3.0, 0.0, 0.0, 1, 1, 1, 1, -1.0, -1.0, 0.0]
iteration=3, obj=-787.67, n violations=9, UB=944.33
 --- 
506.0
[0, 300.0, 300.0, 0, 163.66666666666666, 193.33333333333331, 0, 0, 0, 300.0]
relaxation: [1, -2.0, -2.0, 1, 1, 0.0, 1, 1, 1, -3.0]
iteration=4, obj=-1111.33, n violations=13, UB=863.00
 --- 
825.0
[34.375, 42.666666666666686, 300.0, 0, 300.0, 0, 139.16666666666674, 114.41666666666663, 0, 0]
relaxation: [1, 1, -2.0,

relaxation: [-2.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, -1.0, 1, 1]
iteration=29, obj=353.88, n violations=6, UB=420.92
 --- 
336.0
[36.06763285024158, 19.60966183574888, 71.85652173913033, 53.87198067632846, 54.80628019323674, 42.980676328502355, 40.083574879226944, 11.269565217391204, 46.54202898550724, 65.0371980676328]
relaxation: [-2.0, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 1, 0.0, 1]
iteration=30, obj=332.80, n violations=6, UB=418.05
 --- 
351.99999999999994
[11.203529411764693, 16.969411764705917, 95.7576470588235, 35.297647058823436, 30.65647058823533, 36.98588235294117, 28.63764705882346, 54.039999999999964, 68.76588235294115, 54.01058823529408]
relaxation: [1, 1, -1.0, 1, 0.0, 0.0, 0.0, -1.0, -1.0, 1]
iteration=31, obj=294.92, n violations=7, UB=417.63
 --- 
395.9999999999999
[46.252555622369265, 66.35598316295844, 106.2345159350573, 74.64642212868311, 118.10202445379842, 83.10222489476854, 71.2717979555022, 77.41611545399871, 79.43956704750454, 97.05351773902592]
relaxation: [1, 0.0, 0.0, 0.

relaxation: [0.0, 0.0, 0.0, 0.0, -1.0, 1, 0.0, 0.0, 1, 0.0]
iteration=55, obj=366.62, n violations=3, UB=390.19
 --- 
388.99999999999994
[38.65624999999996, 62.906249999999986, 88.6562499999999, 64.78124999999991, 86.65624999999993, 45.56249999999999, 63.781249999999915, 53.03124999999991, 42.281249999999936, 77.46874999999994]
relaxation: [-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1, 1]
iteration=56, obj=370.44, n violations=4, UB=390.19
 --- 
375.99999999999994
[28.781249999999908, 53.03124999999992, 78.78124999999986, 54.90624999999986, 57.03124999999977, 25.812499999999886, 53.90624999999985, 43.15624999999985, 32.406249999999886, 67.59374999999989]
relaxation: [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1]
iteration=57, obj=369.22, n violations=3, UB=390.19
 --- 
297.9999999999999
[49.75000000000023, 69.4999999999996, 99.75000000000017, 75.87500000000011, 78.00000000000006, 25.812499999999886, 70.3125, 64.12500000000011, 53.37500000000017, 88.5625000000001]
relaxation: [-2.0, 0.0

relaxation: [0.0, 0.0, 1, 0.0, -1.0, 0.0, 0.0, 1, 0.0, 1]
iteration=81, obj=379.00, n violations=4, UB=383.00
 --- 
284.9999999999999
[29.999999999999943, 74.00000000000006, 70.00000000000006, 60.99999999999994, 77.99999999999983, 30.999999999999886, 55.999999999999716, 32.00000000000003, 43.99999999999977, 73.99999999999986]
relaxation: [1, 0.0, 1, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1]
iteration=82, obj=383.00, n violations=4, UB=383.00
Lagrangian dual: 383.0
-- best feasible --
solution for: Lagrangian
objective: 383
open_0=1
open_1=1
open_2=1
open_4=1
supplier_0_4=1
supplier_1_1=1
supplier_2_4=1
supplier_3_0=1
supplier_4_4=1
supplier_5_1=1
supplier_6_1=1
supplier_7_2=1
supplier_8_1=1
supplier_9_2=1

